In [2]:
import random
import math
import numpy as np
import itertools

def demand(t, uncertainity = 0.2):
    dt = 1000*(1+1/2.0*math.sin(math.pi*(t-1)/12))
    return (random.random()-0.5)*2*uncertainity*dt + dt

def cost(t, i):
    factor = [1, 1.5, 2]
    return factor[i]*(1+1/2.0*math.sin(math.pi*(t-1)/12))

T = 24
I = 3
P = 567 #Max factory capacity
Vmin = 500 #Minimum warehouse level of inventory
Vmax = 2000 #Maximum warehouse level of inventory
Q = 13600 // (24/T) #Max factory cumulative capacity
d = np.zeros((T,1))
d_ = np.zeros((T,1))
for i in range(T):
    d[i, 0] = demand(i)
    d_[i, 0] = demand(i, 0) * 1.2
c = np.zeros((I, T))
for i, j in itertools.product(range(I), range(T)):
    c[i,j] = cost(j, i)

In [21]:
from cvxpy import *

def solve(factoryProduced, stock, t):
    production = Variable(I, t)
    costs = np.zeros((I, t))
    dmd = np.zeros((t,1))
    for i in range(t):
        dmd[i,0] = d_[T - t + i, 0]
    for i, j in itertools.product(range(I), range(t)):
        costs[i, j] = c[i, j + (T-t)]
        
    objective = Minimize(trace(production.T * costs))
    constraints = [min_entries(production) >= 0, 
                   max_entries(production) <= P,
                   max_entries(production * np.ones((t, 1))+factoryProduced) <= Q]
    
    for k in range(t):
        parSum = np.zeros((t, 1))
        dParSum = 0
        for i in range(k+1):
            parSum[i, 0] = 1
            dParSum += dmd[i]
        constraints.append(np.ones((1, I)) * (production * parSum) - dParSum + stock >= Vmin)
        constraints.append(np.ones((1, I)) * (production * parSum) - dParSum + stock <= Vmax)

    problem = Problem(objective, constraints)
    result = problem.solve()
    
    #DEBUG
    #print production.value    
    #for k in range(t):
    #    parSum = np.zeros((t, 1))
    #    dParSum = 0
    #    for i in range(k+1):
    #        parSum[i, 0] = 1
    #        dParSum += dmd[i]
    #    print (np.ones((1, I)) * (production.value * parSum) + stock - dParSum)
    #end DEBUG
    return (production.value)[:,0]

def plan():
    stock = 0
    produced = np.zeros((3, 1))
    for i in range(T):
        month = solve(np.zeros((3, 1)), stock, T-i)
        produced += month
        for j in range(I):
            stock += month[j, 0]
        stock -= d[i, 0]
        monthReport = [round(x[0,0]) for x in month]
        print "In month " + str(i+1) + " we produced " + str(monthReport) + " pcs"
        print "We ended with a " + str(round(stock)) + "pcs on stock."

plan()

In month 1 we produced [567.0, 567.0, 567.0] pcs
We ended with a 982.0pcs on stock.
In month 2 we produced [567.0, 567.0, 567.0] pcs
We ended with a 1753.0pcs on stock.
In month 3 we produced [567.0, 567.0, 468.0] pcs
We ended with a 2049.0pcs on stock.
In month 4 we produced [567.0, 567.0, 317.0] pcs
We ended with a 2023.0pcs on stock.
In month 5 we produced [567.0, 567.0, 467.0] pcs
We ended with a 2229.0pcs on stock.
In month 6 we produced [567.0, 567.0, 356.0] pcs
We ended with a 2391.0pcs on stock.
In month 7 we produced [567.0, 567.0, 46.0] pcs
We ended with a 2016.0pcs on stock.
In month 8 we produced [567.0, 567.0, 0.0] pcs
We ended with a 1642.0pcs on stock.
In month 9 we produced [567.0, 567.0, 0.0] pcs
We ended with a 1359.0pcs on stock.
In month 10 we produced [567.0, 567.0, 0.0] pcs
We ended with a 887.0pcs on stock.
In month 11 we produced [567.0, 567.0, 103.0] pcs
We ended with a 573.0pcs on stock.
In month 12 we produced [567.0, 567.0, 293.0] pcs
We ended with a 735.0pc